In [1]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

     |████████████████████████████████| 106 kB 22.8 MB/s 
     |████████████████████████████████| 109 kB 43.6 MB/s 
     |████████████████████████████████| 75 kB 3.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 1.5.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-translate 1.5.0 requires google-cloud-core<2.0dev,>=1.0.0, but you have google-cloud-core 2.2.1 which is incompatible.
google-cloud-language 1.2.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-firestore 1.7.0 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-firestore 1.7.0 requires google-cloud-core<2.0dev,>=1.0.3, but you have google-cloud-core 2.2.1 whic

In [3]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [4]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2022-01-07 17:56:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  77.4MB/s    in 0.2s    

2022-01-07 17:56:31 (77.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [5]:
#TODO: sign up for an ngrok account then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 23JsrQWoovIqKcwTbcDwQMmpaX0_4fisNQVeiNo3QKpjqf4tX

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
# install a ngrok python package and a version of flask that works with it in colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

     |████████████████████████████████| 83 kB 1.2 MB/s 


**OUR INSTALLS**


In [7]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq


     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 58.6 MB/s 
     |████████████████████████████████| 93 kB 1.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 1.5.0 requires google-cloud-core<2.0dev,>=1.0.0, but you have google-cloud-core 2.2.1 which is incompatible.
google-cloud-firestore 1.7.0 requires google-cloud-core<2.0dev,>=1.0.3, but you have google-cloud-core 2.2.1 which is incompatible.
google-cloud-datastore 1.8.0 requires google-cloud-core<2.0dev,>=1.0.0, but you have google-cloud-core 2.2.1 which is incompatible.
google-cloud-bigquery 1.21.0 requires google-cloud-core<2.0dev,>=1.0.3, but you have google-cloud-core 2.2.1 which is incompatible.
google-cloud-bigquery 1.21.0 requires google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you have google-resumable-media 2.

In [8]:
from collections import Counter, OrderedDict
from google.cloud import storage
from google.colab import auth
from graphframes import *
from inverted_index_colab import *
from itertools import islice, count, groupby
from nltk.corpus import stopwords
from nltk.stem.porter import *
from operator import itemgetter
from pathlib import Path
from pyspark import SparkContext, SparkConf
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from time import time
from timeit import timeit
import builtins
import hashlib
import itertools
import math
import nltk
import numpy as np
import os
import pandas as pd
import pickle
import pyspark
import re
import sys

**index creation**

In [9]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar


--2022-01-07 17:57:58--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 52.85.146.59, 52.85.146.12, 52.85.146.3, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|52.85.146.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247880 (242K) [binary/octet-stream]
Saving to: ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’

graphframes-0.8.2-s 100%[===================>] 242.07K  --.-KB/s    in 0.006s  

2022-01-07 17:57:58 (36.8 MB/s) - ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ saved [247880/247880]



In [10]:
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
/ [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [11]:
from pathlib import Path 
try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)

doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd
doc_anchor_text_before = parquetFile.limit(1000).select("id","anchor_text").rdd
doc_anchor_text_pairs = (doc_anchor_text_before.map(lambda x: (map(lambda y: y[1],x[1]),x[0]))).map(lambda x: (" ".join(x[0]),x[1]))

In [12]:
from inverted_index_colab import *
!mkdir text title anchor

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)


def doc_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  countTokens = OrderedDict(Counter(tokens))
  dict_keys = list(countTokens.keys())
  for token in dict_keys:
    if token in all_stopwords:
      countTokens.pop(token)
  countTokens_len = builtins.sum(countTokens.values())
  return (id,countTokens_len)

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  countTokens = OrderedDict(Counter(tokens))
  dict_keys = list(countTokens.keys())
  for token in dict_keys:
    if token in all_stopwords:
      countTokens.pop(token)
  return list(map(lambda x: (x , (id,countTokens[x])),countTokens.keys()))


def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl)

def calculate_df(postings):
  return postings.map(lambda x: (x[0],len(x[1])))


NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings,basedir):
  newpostings = postings.map(lambda x: (token2bucket_id(x[0]),x))
  words2buckets = newpostings.groupByKey()
  x=words2buckets.map(lambda x: InvertedIndex.write_a_posting_list(x,basedir))
  return x
def countForDict(posting_locs_list):
  super_posting_locs = defaultdict(list)
  for posting_loc in posting_locs_list:
    for k, v in posting_loc.items():
      super_posting_locs[k].extend(v)
  return super_posting_locs


def createIndex(doc_rdd,directory):
  inverted = InvertedIndex()
  word_counts = doc_rdd.flatMap(lambda x: word_count(x[0], x[1]))
  postings = word_counts.groupByKey().mapValues(reduce_word_counts)
  postings_filtered = postings.filter(lambda x: len(x[1])>10)
  df_ = calculate_df(postings_filtered)
  inverted.df = df_.collectAsMap()
  word_counter = postings.map(lambda x: (x[0], builtins.sum([y[1] for y in x[1]])))
  inverted.term_total = Counter(word_counter.collectAsMap())
  posting_locs_list = partition_postings_and_write(postings_filtered,directory).collect()
  inverted.posting_locs = countForDict(posting_locs_list)
  return inverted

def writeIdx(inverted, directory, name):
  inverted.write_index(directory, name) 
  inverted.write_dct(directory, name+'_dct') 
 
 

In [13]:
from nltk.stem.porter import *
from BM25_from_index import *

stemmer = PorterStemmer()
!mkdir search_text search_title

def improved_tokenize(text):
    list_of_tokens =  [stemmer.stem(token.group()) for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords] ##is it work?
    return list_of_tokens


def improved_word_count(text, id):
  tokens = [stemmer.stem(token.group()) for token in RE_WORD.finditer(text.lower())]
  countTokens = OrderedDict(Counter(tokens))
  dict_keys = list(countTokens.keys())
  for token in dict_keys:
    if token in all_stopwords:
      countTokens.pop(token)
  return list(map(lambda x: (x , (id,countTokens[x])),countTokens.keys()))

def improved_reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[1], reverse=True)[:100]


def createImprovedIndex(doc_rdd,directory):
  inverted = InvertedIndex()
  word_counts = doc_rdd.flatMap(lambda x: word_count(x[0], x[1]))
  postings = word_counts.groupByKey().mapValues(improved_reduce_word_counts)
  # #postings_filtered = postings.filter(lambda x: len(x[1])>10)
  df_ = calculate_df(postings)
  inverted.df = df_.collectAsMap()
  word_counter = postings.map(lambda x: (x[0], builtins.sum([y[1] for y in x[1]])))
  inverted.term_total = Counter(word_counter.collectAsMap())
  posting_locs_list = partition_postings_and_write(postings,directory).collect()  ##collect? 
  inverted.posting_locs = countForDict(posting_locs_list)
  return inverted



def search_back(clean_query):
  idx_title = InvertedIndex.read_index('search_title', 'index_title')
  idx_text = InvertedIndex.read_index('search_text', 'index_text')
  #topNtitle = get_topN_score_for_queries(clean_query,idx_title)
  #topNtext = get_topN_score_for_queries(clean_query,idx_text)
  dct_title = InvertedIndex.read_index('search_title', 'index_title_dct')
  DL_title = dict(map(lambda tup: (tup[0],len(tup[1].split(' '))),dct_title.items()))
  DL_text = InvertedIndex.read_index('search_text', 'index_text_dct')
  bm25title = BM25_from_index(idx_title,DL_title)
  bm25text = BM25_from_index(idx_text,DL_text)
  title_candidates = bm25title.search(clean_query)
  text_candidates = bm25text.search(clean_query)
  best_text_title = merge_results(title_candidates , text_candidates , N=100)
  return list(map(lambda tup: (tup[0], dct_title[tup[0]]) , best_text_title))
  

def merge_results(title_scores,body_scores,title_weight=0.5,text_weight=0.5,N = 3):    
    # YOUR CODE HERE
    title_dict = dict(title_scores)
    body_dict = dict(body_scores)
    docs_id_for_query = set(list(title_dict.keys()) + list(body_dict.keys()))
    allscores = []
    for docid in docs_id_for_query:
      allscores.append((docid,title_dict.get(docid,0)*title_weight + body_dict.get(docid,0)*text_weight))
    score_sorted = sorted(allscores,key=lambda x:x[1],reverse = True)[:N]
    return score_sorted

In [22]:
title_index = createIndex(doc_title_pairs,'title')
title_index.dct=doc_title_pairs.map(lambda x: (x[1],x[0])).collectAsMap()
writeIdx(title_index, 'title','index_title')


text_index = createIndex(doc_text_pairs,'text')
word_counts_length = doc_text_pairs.map(lambda x: doc_count(x[0], x[1])) #only body
text_index.dct = word_counts_length.collectAsMap()  #only body
writeIdx(text_index, 'text','index_text')


anchor_index = createIndex(doc_anchor_text_pairs,'anchor')
writeIdx(anchor_index, 'anchor','index_anchor')

# title_index = createImprovedIndex(doc_title_pairs,'search_title')
# title_index.dct=doc_title_pairs.map(lambda x: (x[1],x[0])).collectAsMap()
# writeIdx(title_index, 'search_title','index_title')

# text_index = createImprovedIndex(doc_text_pairs,'search_text')
# word_counts_length = doc_text_pairs.map(lambda x: doc_count(x[0], x[1])) #only body
# text_index.dct = word_counts_length.collectAsMap()  #only body
# writeIdx(text_index, 'search_text','index_text')

# Run the app

In [15]:
# you need to upload your implementation of search_app.py
import search_frontend as se
import search_backend as sb

In [16]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(sb)

In [23]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app)
se.app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://476e-35-245-125-86.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
 * Running on http://476e-35-245-125-86.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Jan/2022 18:01:02] "GET /search_body?query=dog HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2022 18:02:02] "GET /search_body?query=data+science HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2022 18:02:09] "GET /search_body?query=data+science HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2022 18:02:37] "GET /search_body?query=data+science HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2022 18:02:40] "GET /search?query=data+science HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2022 18:03:32] "GET /search_body?query=best+marvel+movie HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2022 18:03:36] "GET /search?query=best+marvel+movie HTTP/1.1" 200 -


# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [18]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [19]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [20]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://b4e3-34-86-221-19.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_body', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))

In [21]:
#print(res.text)
print(qs_res)

[('python', 0.09722089767456055, None), ('data science', 0.08135032653808594, None), ('migraine', 0.08765363693237305, None), ('chocolate', 0.07992172241210938, None), ('how to make pasta', 0.0807340145111084, None), ('Does pasta have preservatives?', 0.07966375350952148, None), ('how google works', 0.07768821716308594, None), ('what is information retrieval', 0.08137679100036621, None), ('NBA', 0.07975387573242188, None), ('yoga', 0.07917547225952148, None), ('how to not kill plants', 0.08975553512573242, None), ('masks', 0.08062624931335449, None), ('black friday', 0.0788264274597168, None), ('why do men have nipples', 0.0788261890411377, None), ('rubber duck', 0.07910823822021484, None), ('michelin', 0.07982730865478516, None), ('what to watch', 0.0831899642944336, None), ('best marvel movie', 0.0806124210357666, None), ('how tall is the eiffel tower', 0.07882547378540039, None), ('where does vanilla flavoring come from', 0.08446311950683594, None), ('best ice cream flavour', 0.0822